In [0]:
## Implementing RBM
import numpy as np
import pandas as pd
import torch
import torch.nn as nn     # for building neural network
import torch.nn.parallel     # for parallel computing
import torch.optim as optim    # for optimisation
import torch.utils.data   
from torch.autograd import Variable  # for stocastic gradient descent

In [0]:
df = pd.read_csv('BX-Book-Ratings.csv', names=["User-ID", "ISBN", "Book-Rating"], delimiter=";")

df_mat = df.as_matrix()[1:]

df_mat[:,0] = df_mat[:,0].astype('int64')
df_mat[:,2] = df_mat[:,2].astype('int64')

usercount = (len(np.unique(df_mat[:,0])))
bookcount = (len(np.unique(df_mat[:,1])))



In [0]:
rating_matrix = np.ndarray(shape=(10000,10000), dtype = 'int64')
rating_matrix = np.zeros(shape=(10000,10000))


In [0]:
rating_matrix_test = np.zeros(shape=(10000,10000))
rating_matrix_test = np.ndarray(shape=(10000,10000), dtype = 'int64')

In [0]:
bookid = 0
usernum = 0
bookmap = {}
usermap = {}
useridslist = []
for k in range(0,500000):
    userid = df_mat[k,0]
    isbn = df_mat[k,1]
    if isbn not in bookmap:
        if bookid < 10000:
            bookmap[isbn] = bookid
            if userid not in usermap:
                if usernum < 10000:
                    usermap[userid]=usernum
                    rating_matrix[usernum, bookid] = df_mat[k,2]
                    usernum+=1
            else:
                rating_matrix[usermap[userid], bookid] = df_mat[k,2]
            bookid = bookid+1
    else:
        if userid not in usermap:
            if usernum < 10000:
                usermap[userid]=usernum
                rating_matrix[usernum, bookmap[isbn]] = df_mat[k,2]
                usernum+=1
        else:
            rating_matrix[usermap[userid], bookmap[isbn]] = df_mat[k,2]
            

In [0]:
usertestmap = {}
booktestmap = {}
uid = 0 
iid = 0

In [0]:
bookid = 0
usernum = 0
booktestmap = {}
usertestmap = {}
for k in range(700000, 1000000):
    userid = df_mat[k,0]
    isbn = df_mat[k,1]
    if isbn not in booktestmap:
        if bookid < 10000:
            booktestmap[isbn] = bookid
            if userid not in usertestmap:
                if usernum < 10000:
                    usertestmap[userid]=usernum
                    rating_matrix_test[usernum, bookid] = df_mat[k,2]
                    usernum+=1
            else:
                rating_matrix_test[usertestmap[userid], bookid] = df_mat[k,2]
            bookid = bookid+1
    else:
        if userid not in usertestmap:
            if usernum < 10000:
                usertestmap[userid]=usernum
                rating_matrix_test[usernum, booktestmap[isbn]] = df_mat[k,2]
                usernum+=1
        else:
            rating_matrix_test[usertestmap[userid], booktestmap[isbn]] = df_mat[k,2]

In [0]:
print rating_matrix_test

[[10  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 0  0  0 ...,  0  0  0]]


In [0]:
# converting the data into torch tensors
rating_matrix = torch.FloatTensor(rating_matrix)


In [0]:
# converting ratings to binary: rating > 6 is liked = 1, otw not liked
# the original 0's in the training set are given a value of -1

rating_matrix[rating_matrix == 0] = -1

rating_matrix[rating_matrix == 1] = 0
rating_matrix[rating_matrix == 2] = 0
rating_matrix[rating_matrix == 3] = 0
rating_matrix[rating_matrix == 4] = 0
rating_matrix[rating_matrix == 5] = 0
rating_matrix[rating_matrix == 6] = 0
rating_matrix[rating_matrix == 7] = 0

# converting all ratings > 7 to 1
rating_matrix[rating_matrix >= 8] = 1


In [0]:
rating_matrix_test = torch.FloatTensor(rating_matrix_test)

rating_matrix_test[rating_matrix_test == 0] = -1
rating_matrix_test[rating_matrix_test == 1] = 0
rating_matrix_test[rating_matrix_test == 2] = 0
rating_matrix_test[rating_matrix_test == 3] = 0
rating_matrix_test[rating_matrix_test == 4] = 0
rating_matrix_test[rating_matrix_test == 5] = 0
rating_matrix_test[rating_matrix_test == 6] = 0
rating_matrix_test[rating_matrix_test == 7] = 0
rating_matrix_test[rating_matrix_test >= 8] = 1

In [0]:
# creating the RBM neural network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)   #initialising weights in torch
        self.a = torch.randn(1, nh)   # to initialise the RBM object
        self.b = torch.randn(1, nv)   # for bias
    
    # use gibbs sampling to improve log likelihood gradient
    # sample_h : sample prob of hidden nodes, given visible nodes
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())  # multiply w(tensor weights) and x(visible neurons)
        activ = wx + self.a.expand_as(wx) # to add bias to every row of neuron
        prob_h_given_v = torch.sigmoid(activ)
        return prob_h_given_v, torch.bernoulli(prob_h_given_v) # returns prob and neurons activated by this sampling
    
    # sample v: prob of visible nodes, given hidden nodes
    # estimating prob of visible nodes
    def sample_v(self,y):
        wy = torch.mm(y, self.W)  # multiply w(tensor weights) and x(visible neurons)
        activ = wy + self.b.expand_as(wy) # to add bias to every row of neuron
        prob_v_given_h = torch.sigmoid(activ)
        return prob_v_given_h, torch.bernoulli(prob_v_given_h) # returns prob and neurons activated by this sampling
    
    def train(self,v0, vk, ph0, phk):  # Using k-step contrastive divergence
    #v0:input vec having ratings of all movies by 1 user, vk: visible nodes obtained after k iterations, 
    #ph0: prob of hidden nodes= 1 in first iteration , phk: prob of hidden nodes after k sampling
        self.W += torch.mm(v0.t(),ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0-vk),0) 
        self.a += torch.sum((ph0-phk),0)
        
    

In [0]:
nv = len(rating_matrix[0]) # no. of visible nodes
print nv
nh = 100 # number of features we want to detect

# batch size: to update weights after several iterations and not afer each one
batch_size = 1000
rbm = RBM(nv,nh)


10000


In [0]:
n_iter = 5
for iter in range(0, n_iter):
    train_loss = 0
    rms_train_loss = 0
    counter = 0.
    for user in range(0, 10000 - batch_size, batch_size):
        vk = rating_matrix[user:user+batch_size]# op of gibbs sampling afte k iterations of random walk
        v0 = rating_matrix[user:user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):  # for k steps of contrastive div
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0]= v0[v0<0] # to eliminate -1 ratings
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0,vk,ph0,phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) #MAE
        rms_train_loss += torch.mean((v0[v0>=0] - vk[v0>=0])**2) #RMSE
        #train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) #MAE
        counter += 1.
    print "MAE  on training for iter #"+str(iter)+" is: "+str(train_loss/counter)
    print "RMSE on training for iter #"+str(iter)+" is: "+str((rms_train_loss/counter)**0.5)
    print ""
        
        
            
        

MAE  on training for iter #0 is: 0.380275072065
RMSE on training for iter #0 is: 0.616664472842

MAE  on training for iter #1 is: 0.377912293285
RMSE on training for iter #1 is: 0.614745714328

MAE  on training for iter #2 is: 0.375525298138
RMSE on training for iter #2 is: 0.612801189733

MAE  on training for iter #3 is: 0.370339020544
RMSE on training for iter #3 is: 0.608554862394

MAE  on training for iter #4 is: 0.371679191732
RMSE on training for iter #4 is: 0.609654977616



In [0]:

test_loss = 0
counter = 0.
mae_test_loss = 0
rmse_test_loss = 0

for user in range(10000):
    v = rating_matrix[user:user+1]
    vt = rating_matrix_test[user:user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
    if len(vt[vt>=0]) > 0 and len(v[vt>=0]) > 0:
        mae_test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) #MAE
        rmse_test_loss += torch.mean((vt[vt>=0] - v[vt>=0])**2) #RMSE
    counter += 1.
print "MAE  on testing is: "+str(mae_test_loss/counter)
print "RMSE on testing is: "+str((rmse_test_loss/counter)**0.5)
        


MAE  on training for iter #4 is: 0.347591827294
RMSE on training for iter #4 is: 0.589569187877
